## Dependencies

In [8]:
import xlrd
from ortools.sat.python import cp_model
from ortools.linear_solver import pywraplp
from itertools import product
from flask import Flask, jsonify, Response
from itertools import product
import pandas as pd

import os

## Utility and Solver Functions

In [3]:

# read data
def read_data(path, sheetIndex=0):
    names, coords, supply, demand = [], [], [], []
    wb = xlrd.open_workbook(path)
    sheet = wb.sheet_by_index(sheetIndex)
    for i in range(1, sheet.nrows):
        names.append(sheet.cell_value(i, 0))
        supply.append(sheet.cell_value(i, 1))
        demand.append(sheet.cell_value(i, 2))
        coords.append((sheet.cell_value(i, 3), sheet.cell_value(i, 4)))
    return names, coords, supply, demand


def get_target_values(supply, demand, node=None):
    adjusted_supply = [s - min(d, s) for s, d in zip(supply, demand)]
    adjusted_demand = [d - min(d, s) for s, d in zip(supply, demand)]

    supply_sum = float(sum(adjusted_supply))
    demand_sum = float(sum(adjusted_demand))

    supply_ratio = min(demand_sum / supply_sum, 1.0)
    if node and adjusted_demand[node] <= supply_sum:
        demand_ratio = min((supply_sum - adjusted_demand[node]) / (demand_sum - adjusted_demand[node]), 1.0)
    else:
        demand_ratio = min(supply_sum/demand_sum, 1.0)

    sources = [s * supply_ratio for s in adjusted_supply]
    targets = [d * demand_ratio for d in adjusted_demand]
    if node: 
        targets[node] = adjusted_demand[node]
        
    return sources, targets


def buid_optimization_model(sources, targets, distance_matrix):
    solver = pywraplp.Solver('ResourceAllocation', pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)
    # sources, targets = get_target_values(supply, demand)

    source_upper_bound = max(sources)
    target_upper_bound = max(targets)

    transfers = {}
    bin_transfers = {}

    for i, j in product(range(len(sources)), range(len(targets))):
        transfers[(i, j)] = solver.NumVar(0, int(source_upper_bound) + 1, 'transfer_%i_to_%i' % (i, j))

    cons1 = {}
    cons2 = {}
    for i in range(len(sources)):
        cons1[i] = solver.Constraint(sources[i], sources[i])
        cons2[i] = solver.Constraint(targets[i], targets[i])
        for j in range(len(targets)):
            cons1[i].SetCoefficient(transfers[(i, j)], 1)
            cons2[i].SetCoefficient(transfers[(j, i)], 1)

    # objective function
    objective = solver.Objective()

    for i in range(len(sources)):
        for j in range(len(targets)):
            objective.SetCoefficient(transfers[(i, j)], distance_matrix[i][j])

    solver.Solve()

    result = {}
    for i, j in product(range(len(sources)), range(len(targets))):
        result[(i, j)] = transfers[(i, j)].solution_value()

    return result


## Use Case - Global Optimal Allocations

In [35]:

path = 'Food_Allocation.xlsx'
numDays = 3
dayNum = 1
for dayNum in range(numDays):
    
    names, coords, supply, demand = read_data(path, dayNum)
    nodes_list = [ [i, names[i], coords[i][0], coords[i][1], supply[i], demand[i]] 
                  for i in range(len(names)) ]
    df_nodes = pd.DataFrame(nodes_list, columns = ['Area ID', 'Area Name', 'Latitude', 'Longitude', 
                                            'Daily Stock', 'Daily Demand'])
    #Showing Nodes Info
    print('\n------------------------ Day ', str(dayNum + 1), ' ------------------------')
    print('\n=========================== Area Supply/Demand Info ========================== \n')
    print(df_nodes, '\n')

    focusNode = None
    if focusNode != None:
        focusNode = int(focusNode)

    sources, targets = get_target_values(supply, demand, node=focusNode)

    distance_matrix = [[1 for s in sources] for t in targets]

    result = buid_optimization_model(sources, targets, distance_matrix)

    transfer_list = [ [names[i], names[j], result[(i, j)] ] 
                     for (i, j) in product(range(len(sources)), range(len(targets))) 
                     if result[(i, j)] > 0 ]
    df_transfers = pd.DataFrame(transfer_list, columns = ['From' , 'To' , 'Transfer Qty'])

    #Showing Optimal Transfer Values
    print('\n=========================== Optimal Resource Transfers ========================== \n')
    print(df_transfers)
    print('\n...................................................................................')




------------------------ Day  1  ------------------------

=========================== Area Supply/Demand Info ========================== 

   Area ID          Area Name   Latitude  Longitude  Daily Stock  Daily Demand
0        0         Whitefield  12.580000  77.440000       2000.0        7000.0
1        1        Indiranagar  12.975100  77.640060       7100.0        2600.0
2        2               HRBR  13.020028  77.640062       7900.0        9200.0
3        3                HSR  12.914974  77.649847          0.0        8700.0
4        4    Electronic_City  12.844523  77.644869       7000.0        5500.0
5        5       Banashankari  12.932207  77.552000       6900.0        8700.0
6        6      Yeshwanthpura  13.025045  77.524877          0.0         500.0
7        7         Kormangala  12.944253  77.617059       6700.0        8000.0
8        8          Jayanagar  12.934549  77.579293       6900.0        2500.0
9        9  Basaweshwaranagar  12.989628  77.535862       8300.0     

## Use Case - Optimal Resource Allocation with a Single Node in Emergency State

In [37]:
path = 'Food_Allocation.xlsx'
#day 0-2
dayNum = 0

#Focused Node (0-9)
focusNode = 7
    
names, coords, supply, demand = read_data(path, dayNum)
nodes_list = [ [i, names[i], coords[i][0], coords[i][1], supply[i], demand[i]] 
                  for i in range(len(names)) ]
df_nodes = pd.DataFrame(nodes_list, columns = ['Area ID', 'Area Name', 'Latitude', 'Longitude', 
                                            'Daily Stock', 'Daily Demand'])
#Showing Nodes Info
print('\n------------------------ Day ', str(dayNum + 1), ' ------------------------')
print('\n=========================== Area Supply/Demand Info ========================== \n')
print(df_nodes, '\n')


if focusNode != None:
    focusNode = int(focusNode)

sources, targets = get_target_values(supply, demand, node=focusNode)

distance_matrix = [[1 for s in sources] for t in targets]

result = buid_optimization_model(sources, targets, distance_matrix)

transfer_list = [ [names[i], names[j], result[(i, j)] ] 
                     for (i, j) in product(range(len(sources)), range(len(targets))) 
                     if result[(i, j)] > 0 ]
df_transfers = pd.DataFrame(transfer_list, columns = ['From' , 'To' , 'Transfer Qty'])

    #Showing Optimal Transfer Values
print('\n=========================== Optimal Resource Transfers ========================== \n')
print(df_transfers)
print('\n...................................................................................')


------------------------ Day  1  ------------------------

=========================== Area Supply/Demand Info ========================== 

   Area ID          Area Name   Latitude  Longitude  Daily Stock  Daily Demand
0        0         Whitefield  12.580000  77.440000       2000.0        7000.0
1        1        Indiranagar  12.975100  77.640060       7100.0        2600.0
2        2               HRBR  13.020028  77.640062       7900.0        9200.0
3        3                HSR  12.914974  77.649847          0.0        8700.0
4        4    Electronic_City  12.844523  77.644869       7000.0        5500.0
5        5       Banashankari  12.932207  77.552000       6900.0        8700.0
6        6      Yeshwanthpura  13.025045  77.524877          0.0         500.0
7        7         Kormangala  12.944253  77.617059       6700.0        8000.0
8        8          Jayanagar  12.934549  77.579293       6900.0        2500.0
9        9  Basaweshwaranagar  12.989628  77.535862       8300.0     